In [3]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import keras


In [4]:
def eval_keras(df_name: str, model_name: str, threshold:float) -> list:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # load the saved model
    model = load_model(model_name)

    # read the data for the predictions to be compared to
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the test data and get recommended bets
    winning_bets = []
    losing_bets = []
    for i in range(len(real_X)):
        try:
            prediction = model.predict(scaler.transform(real_X.iloc[i].values.reshape(1, -1)), verbose=0)
            actual = real_y.iloc[i]
            if prediction > threshold and actual == 1:
                winning_bets.append([real_X.iloc[i].loc['Column501'], prediction.item()])
            elif prediction > threshold and actual == 0:
                losing_bets.append([real_X.iloc[i].loc['Column501'], prediction.item()])
        except:
            pass
    return [winning_bets, losing_bets]

In [5]:
import torch
import torch.nn as nn


def eval_pytorch(df_name: str, model_name: str, threshold:float) -> list:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)

        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # load the saved model
    model = ANN_Model(input_size=X_train.shape[1])
    model.load_state_dict(torch.load(model_name))
    device = torch.device("cuda")
    model.to(device)
    model.eval()

    # loop through the test data and get recommended bets
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the test data and get recommended bets
    winning_bets = []
    losing_bets = []
    for i in range(len(real_X)):
        X_tensor = torch.tensor(scaler.transform(np.array(real_X.iloc[i].values.reshape(1, -1))), dtype=torch.float32).cuda().to(device)
        y_tensor = torch.tensor([real_y.iloc[i]], dtype=torch.float32).cuda().to(device)
        outputs = model(X_tensor)
        prediction = torch.sigmoid(outputs).item()
        actual = real_y.iloc[i]

        if prediction > threshold and actual == 1:
            winning_bets.append([real_X.iloc[i].loc['Column501'], prediction])
        elif prediction > threshold and actual == 0:
            losing_bets.append([real_X.iloc[i].loc['Column501'], prediction])

    return [winning_bets, losing_bets]


In [6]:
def calculate_winnings(bet_amount, odds):
    if odds > 0:
        return (bet_amount * odds / 100)
    return bet_amount / (odds / -100)


def american_to_decimal(american_odds):
    if american_odds >= 100:
        decimal_odds = (american_odds / 100)
    else:
        decimal_odds = (100 / abs(american_odds))
    return round(decimal_odds, 2)


def calculate_profit(bets:list):
    winnings = 0
    losings = 0
    for bet in bets[0]:
        odds = bet[0]
        winnings += calculate_winnings(100, odds)
    for bet in bets[1]:
        losings += calculate_winnings(100, odds)
    return winnings - losings

In [15]:
threshold_list = {}
for i in range(9900, 9999):
    bets = eval_pytorch("csvs/new_ml_X.csv", "built_models/torch_ml.pth", (i / 10000))
    investment = (len(bets[0]) + len(bets[1])) * 100
    profit = calculate_profit(bets) / investment
    threshold_list[i] = profit
    print(i, investment, profit)

9900 9900 0.16851789369420903
9901 9700 0.2991151663824782
9902 9900 0.1415013514602755
9903 10000 0.17863066929300525
9904 9600 0.1538177006207695
9905 10100 0.1696874182984268
9906 10100 0.1696874182984268
9907 9900 0.11459669412083562
9908 9700 0.13704074474494857
9909 9300 0.22800088358203288
9910 10100 0.1375753190546432
9911 9800 0.08265668218692888
9912 8700 0.23720214641812634
9913 9700 0.1651546946428518
9914 9000 0.2252493453345965
9915 9000 0.2252493453345965
9916 10300 0.13810281033724373
9917 9300 0.24308632561244647
9918 9000 0.23654635663160778
9919 9100 0.228714083743453
9920 9200 0.22086006979257247
9921 8900 0.233130691643867
9922 9100 0.21754121542772756
9923 9000 0.23654635663160778
9924 8800 0.2290954120703013
9925 8500 0.2539878551853876
9926 8800 0.21480631574995496
9927 8900 0.24435615561222584
9928 8700 0.24943692538718526
9929 9100 0.228714083743453
9930 8600 0.24549740947171475
9931 8600 0.2578744533125069
9932 8500 0.2391944378184408
9933 8700 0.234983586580

In [16]:
threshold_list = {}
for i in range(50, 100):
    bets = eval_keras("csvs/new_ml_X.csv", "built_models/keras_ml.h5", (i / 100))
    investment = (len(bets[0]) + len(bets[1])) * 100
    profit = calculate_profit(bets) / investment
    threshold_list[i] = profit
    print(i, investment, profit)

50 61100 0.15505435838466708
51 57300 0.15190260311109421
52 52400 0.1718171337510168
53 46400 0.17528736301027947
54 41500 0.1678274904140346
55 36700 0.16813151002353813
56 34000 0.1561394701367648
57 31000 0.14643287857278
58 27700 0.1629642113319983
59 24000 0.16367195521769665
60 21700 0.15664300072771092
61 19200 0.16089115918599683
62 17000 0.1696685892293021
63 15600 0.16700872595500332
64 13800 0.15964852911872607
65 12300 0.129830511602786
66 10200 0.23163307843966982
67 8900 0.20935821186942832
68 7100 0.2086419007191932
69 6500 0.22396100401245853
70 5500 0.29673061106147064
71 4500 0.28990365949159747
72 4300 0.28033050431788126
73 3100 0.14686898326562675
74 3000 0.1287761102940212
75 2500 0.11922237396986325
76 2100 0.05843276704621243
77 2100 0.05843276704621243
78 1700 -0.033847121736395105
79 1600 -0.004712566844919799
80 1400 0.06604278074866309
81 1100 0.15592286501377406
82 800 0.32803030303030295
83 600 0.5383838383838383
84 600 0.5383838383838383
85 600 0.5383838

In [17]:
a = eval_keras("csvs/new_ml_X.csv", "built_models/keras_ml.h5", .69)
print(len(a[0]), len(a[1]))
print(a)

40 22
[[[-350.0, 0.7725968956947327], [-350.0, 0.7726003527641296], [165.0, 0.9222936630249023], [155.0, 0.92403244972229], [-205.0, 0.70369553565979], [-110.0, 0.7314742207527161], [-300.0, 1.1279841661453247], [-330.0, 1.1379417181015015], [-240.0, 0.6900835037231445], [-125.0, 0.7044974565505981], [-170.0, 0.7059546113014221], [-185.0, 0.7520700097084045], [-164.0, 0.7462430000305176], [-180.0, 0.7297868728637695], [-180.0, 0.7297911047935486], [-315.0, 0.7536284923553467], [-350.0, 0.760481059551239], [-165.0, 0.695496678352356], [-160.0, 0.7667893171310425], [-165.0, 0.7685196995735168], [-120.0, 0.7271961569786072], [-110.0, 0.7250921726226807], [-170.0, 0.8038197755813599], [-200.0, 0.8047020435333252], [-105.0, 0.7003533840179443], [-110.0, 0.7013684511184692], [-110.0, 0.7218671441078186], [-113.0, 0.7219878435134888], [-315.0, 0.7413793802261353], [-310.0, 0.7400794625282288], [-130.0, 0.7111746072769165], [-110.0, 0.6984044313430786], [-165.0, 0.7415156364440918], [-170.0, 0

In [20]:
b = eval_pytorch("csvs/new_ml_X.csv", "built_models/torch_ml.pth", .9975)
print(len(b[0]), len(b[1]))
print(b)

42 14
[[[-170.0, 0.9976100921630859], [-205.0, 0.998725950717926], [-350.0, 0.9996789693832397], [-350.0, 0.9996790885925293], [165.0, 0.9997981190681458], [155.0, 0.9998173117637634], [-170.0, 0.9996016621589661], [-200.0, 0.9997205138206482], [-145.0, 0.9996823072433472], [-170.0, 0.9997621178627014], [-180.0, 0.9995967745780945], [-190.0, 0.9996277093887329], [-190.0, 0.9981676340103149], [-190.0, 0.9981679916381836], [-185.0, 0.9995373487472534], [-190.0, 0.9995587468147278], [-195.0, 0.9979648590087891], [-185.0, 0.9978006482124329], [-220.0, 0.9993792772293091], [-190.0, 0.9989758729934692], [-160.0, 0.9992951154708862], [-150.0, 0.9992378950119019], [-180.0, 0.9995835423469543], [-162.0, 0.9992961883544922], [-120.0, 0.9999436140060425], [-120.0, 0.9999436140060425], [-125.0, 0.9983470439910889], [-117.0, 0.9981895089149475], [-310.0, 0.9998974800109863], [-270.0, 0.9998738765716553], [-185.0, 0.9991903901100159], [-180.0, 0.999091625213623], [-110.0, 0.9984608888626099], [-111.